Import modules and create dataframes from each sheet from Excel file. 

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

murdock_heartfailure = pd.read_excel("Murdock_HeartFailure.xlsx",sheet_name=None, engine="openpyxl")
participants = murdock_heartfailure["Participants"]
labs = murdock_heartfailure["Labs"]
vitals = murdock_heartfailure["Vitals"]
followup = murdock_heartfailure["FollowUp"]
questionnaire = murdock_heartfailure["Questionnaire"]
problemlist = murdock_heartfailure["ProblemList"]

The table below displays the demographic breakdown by sex, age, and race for each cohort.

In [ ]:
sex_summary = participants.groupby("Cohort")["Sex"].value_counts().unstack()
age_summary = participants.groupby("Cohort")["Age"].describe().unstack()

race_columns = ["White", "Black", "Asian", "AmericanIndian", "PacificIslander", "Other"]
participants["race_count"] = participants[race_columns].notna().sum(axis=1)
#function to assign participant to a "Mixed" group if race_count is >1
def assign_race(row):
    if row["race_count"]> 1:
        return "Mixed"
    for race in race_columns:
        if pd.notna(row[race]):
            return race
    return "Unknown"
#create a column that assigns a race group to each row
participants["race_group"]= participants.apply(assign_race, axis=1)
race_summary= participants.groupby("Cohort")["race_group"].value_counts().unstack()

mean_age = age_summary["mean"]

summary_table =  pd.concat([sex_summary, age_summary["mean"].rename("Mean Age"), race_summary], axis=1)
print(summary_table)

The table below displays descriptive statistics for age, weight, height, heart rate, systolic and diastolic blood pressure by cohort. Cohort A had no heart failure at serum collection or during follow-up; Cohort B developed heart failure after collection.

In [ ]:
age_summary = participants.groupby("Cohort")["Age"].describe().unstack()
vitals = vitals.merge(participants[["ParticipantID","Cohort"]], on="ParticipantID", how="left")
weight_summary = vitals.groupby("Cohort")["Wt_lbs"].describe().unstack()
height_summary = vitals.groupby("Cohort")["Ht_inches"].describe().unstack()
hr_summary = vitals.groupby("Cohort")["HeartRate"].describe().unstack()
bp_systolic = vitals.groupby("Cohort")["SystolicBP"].describe().unstack()
bp_diastolic = vitals.groupby("Cohort")["DiastolicBP"].describe().unstack()

participant_descriptive_summary = pd.concat([age_summary, weight_summary, height_summary, hr_summary, bp_systolic, bp_diastolic], axis=1, keys=["Age",
    "Weight", "Height", "Heart rate", "Systolic bp", "Diastolic bp"])
print(participant_descriptive_summary)


NTproBNP is the current standard biomarker for ruling in or ruling out heart failure. A cutoff value of >300 used here is the general cutoff value; however, some guidelines suggest using age-related cutoff values. The boxplot displays the distribution of NTproBNP values > 300 between Cohort A and Cohort B. The distribution of values appear comparable between the two cohorts.

In [ ]:
NTproBNP = labs[(labs["Measure"] == "NTproBNP") & (labs["LabValue"] > 300)].copy()

#Create boxplot
fig, ax= plt.subplots(figsize= (6,4))

#Group and plot values
NTproBNP.boxplot(column= "LabValue", by= "Cohort", ax=ax, showfliers=False)

#Format the plot
ax.set_xlabel("Participant groups")
ax.set_ylabel("NTproBNP lab values")
ax.set_title(" ")
fig.suptitle("NTproBNP: Cohort A vs. Cohort B")
ax.grid(False)
plt.tight_layout()


The boxplot below shows the distribution of PNC values for Cohort A and Cohort B. Participants in Cohort A (no heart failure) tend to have lower PNC levels compared to those in Cohort B.

In [ ]:

pnc= labs[(labs["Measure"]== "PNC")].copy()

#create a boxplot
fig, ax= plt.subplots(figsize= (6,4))

#group and plot values
pnc.boxplot(column= "LabValue", by= "Cohort", ax=ax, showfliers= False)

#format the plot
ax.set_xlabel("Participant groups")
ax.set_ylabel("PNC lab values")
ax.set_title(" ")
fig.suptitle("PNC values: Cohort A vs. Cohort B")
ax.grid(False)
plt.tight_layout()

The boxplot below shows the distribution of PNC values among participants with and without a diagnosis of coronary artery disease (a risk factor for heart failure), regardless of cohort assignment. Participants with coronary artery disease tend to exhibit lower PNC levels.

In [ ]:
#create values and labels
cad_participant_id = problemlist.loc[problemlist["Diagnoses"].str.lower()== "coronary artery disease","ParticipantID"].unique()
pnc= labs[(labs["Measure"]== "PNC")].copy()
pnc["has_cad"]= pnc["ParticipantID"].isin(cad_participant_id).map({False: "no coronary artery disease", True: "coronary artery disease"})

#create boxplot grouped by CAD status
fig, ax= plt.subplots(figsize=(6,4))

#group and plot
pnc.boxplot(column="LabValue", by= "has_cad",ax=ax, showfliers= False)

#format the plot
ax.set_xlabel("Paticipant groups")
ax.set_ylabel("PNC lab values")
ax.grid(False)
ax.set_title(" ")
fig.suptitle("PNC lab values: Non-CAD vs. CAD Participants")
plt.tight_layout()

Heart attack is another risk factor for heart failure. The boxplot below shows the distribution of PNC values among participants with and without a history of heart attack, regardless of cohort. Those with a history of heart attack tend to have higher PNC levels.

In [ ]:
heart_attack_participant = problemlist.loc[problemlist["Diagnoses"].str.lower()=="heart attack", "ParticipantID"].unique()
pnc = labs[(labs["Measure"]=="PNC")].copy()
pnc["had_heart_attack"] = pnc["ParticipantID"].isin(heart_attack_participant).map({False:"no heart attack", True:"heart attack"})

#create boxplot grouped by heart attack status
fig, ax= plt.subplots(figsize=(6,4))

#group and plot
pnc.boxplot(column="LabValue", by= "had_heart_attack", ax=ax, showfliers= False)

#format the boxplot
ax.set_xlabel("Participant groups")
ax.set_ylabel("PNC lab values")
ax.set_title(" ")
ax.grid(False)
fig.suptitle("PNC values: heart attack vs no heart attack")
plt.tight_layout()

Lastly, the boxplot below compares the distribution of PNC values between participants with and without high blood pressure. The overall spread indicates that individuals with high blood pressure generally exhibit higher PNC levels.

In [ ]:
participants_htn = problemlist.loc[problemlist["Diagnoses"].str.lower()== "high blood pressure", "ParticipantID"].unique()
pnc = labs[(labs["Measure"]== "PNC")].copy()
pnc["high blood pressure"] = pnc["ParticipantID"].isin(participants_htn).map({False: "no high blood pressure", True:"high blood pressure"})

#create boxplot
fig, ax= plt.subplots(figsize=(6,4))

#group and plot values
pnc.boxplot(column="LabValue", by="high blood pressure", ax=ax, showfliers= False)

#format boxplot
ax.set_ylabel("PNC lab values")
ax.set_xlabel("Participant groups")
ax.set_title(" ")
fig.suptitle("PNC value: HTN vs. no HTN")
ax.grid(False)
plt.tight_layout()